# Version Control (Git)

Los "version control systems" (VCSs) son herramientas que sirven para llevar un historial de los cambios realizados al código fuente (así como archivos y folders). Además también facilita la colaboración. El VCS más común en la actualidad es Git.

## Modelo de Datos de Git

### Snapshots

En Git, a un archivo se le llama "blob" (un grupo de bytes) y a las carpetas que contienen archivos se les llama "árboles".

```json
<root> (tree)
|
+- foo (tree)
|  |
|  + bar.txt (blob, contents = "hello world")
|
+- baz.txt (blob, contents = "git is wonderful")
```

### Modeling History: Relating Snapshots

Al estado de un programa en un momento específico se le denomina "snapshot". Para relacionar diferentes snapshots, Git utiliza un grafo dirigido acíclico (DAG), o un grupo de snapshots que apuntan a versiones previas, predecesores o "padres".

```json
o <-- o <-- o <-- o
            ^
             \
              --- o <-- o
```

A cada nodo en el grafo se le llama "commit". Las flechas apuntan a los "padres" de cada commit. Como se vio antes, el historial de commits puede no siempre ser lineal. A veces puede diverger en ciertos nodos (para acomodar nuevas características por ejemplo) o en algunos momentos reunirse en "merge commits".

```
o <-- o <-- o <-- o <---- o
            ^            /
             \          v
              --- o <-- o
```

Los commits en Git son inmutables. No pueden cambiarse. Los errores pueden corregirse pero las ediciones al commit lo único que hacen es crear un nuevo commit.

### Data Model, as Pseudocode

```
// a file is a bunch of bytes
type blob = array<byte>

// a directory contains named files and directories
type tree = map<string, tree | blob>

// a commit has parents, metadata, and the top-level tree
type commit = struct {
    parent: array<commit>
    author: string
    message: string
    snapshot: tree
}
```

### Objects and Content-addressing

Un objeto puede ser un "blob", arbol o commit. En Git, todos los objetos son utilizados apuntando a su hash SHA-1.

```
objects = map<string, object>

def store(object):
    id = sha1(object)
    objects[id] = object

def load(id):
    return objects[id]
```

Cuando un objeto referencia a otro, estos no guardan una copia de los mismos en memoria sino que hacen uso del hash. Por ejemplo, para el arbol `<root> (tree)` el hash es el siguiente:

```
100644 blob 4448adbf7ecd394f42ae135bbeed9676e894af85    baz.txt
040000 tree c68d233a33c5c06e0340e4c224f0afca87c8ce87    foo
```

Si se revisan los contenidos referenciados por el hash que representa a `baz.txt` se obtiene el siguiente texto (utilizando el comando `git cat-file -p 4448adbf7ecd394f42ae135bbeed9676e894af85`)

`git is wonderful`

### References

Lo malo de identificar snapshots por su hash SHA-1 es que no es práctico recordar un string de 40 números hexadecimales. La solución propuesta por git para esto son las "referencias", nombres legibles por humanos que apuntan a un snapshot específico. Las referencias apuntan a commits y no son inmutables (como los commits). 

En Git, al "presente" o snapshot actual se le denomina "HEAD".

### Repositories

Un repositorio consiste del conjunto de `objetos` y `referencias`. Esto es lo único que Git almacena en memoria. Todos los comandos `git` únicamente manipulan el grafo acíclico (DAG) al agregar objetos y actualizar referencias. 

Por ejemplo, si se desean descartar cambios "uncommited" y luego hacer que el "master" referencie al commit `5d83f9e`, se hace lo siguiente.

```json
checkout master
git reset --hard 5d83f9e
```

### Staging Area 

Git no solo "crea un snapshot" a partir del estado actual de un directorio. Git permite elegir los archivos que se incluirán en el snapshot por medio de la "staging area".

### Git Command-line Interface

#### Basics

- `git help <command>`: Ayuda para un comando.
- `git init`: Crea un nuevo repo con datos guardados en el archivo `.git`.
- `git status`: 

### Ejemplos

Se inicializará un repositorio para poder "trackear" las tareas y proyectos futuros que se realizarán en la clase de "Ciencia de Datos en Python".

In [4]:
# Importa la librería "os"
import os

# Cambia el directorio local para iniciar el repo en este directorio
os.chdir("E:\\Archivos\\Educación\\Posgrado\\Universidad Galileo (UG)\\Ciencia de Datos en Python")

# Imprime el directorio actual
print(os.getcwd())

E:\Archivos\Educación\Posgrado\Universidad Galileo (UG)\Ciencia de Datos en Python


In [5]:
# Inicializa el repositorio en el directorio actual
!git init

Initialized empty Git repository in E:/Archivos/Educación/Posgrado/Universidad Galileo (UG)/Ciencia de Datos en Python/.git/


In [6]:
# Notifica sobre archivos no "stageados" y archivos no trackeados en el master.
!git status

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.vscode/
	Lectures/
	Literatura/
	Tareas/

nothing added to commit but untracked files present (use "git add" to track)


In [8]:
# "Stagea" un archivo para un commit
!git add "Tareas/Tarea No. 2 - Resumen Git.ipynb"

The file will have its original line endings in your working directory


In [9]:
# Hace un commit
!git commit 

[master (root-commit) 3c560a7] Jupyter notebook de la tarea no 2
 1 file changed, 281 insertions(+)
 create mode 100644 Tareas/Tarea No. 2 - Resumen Git.ipynb


In [10]:
# Muestra los commits realizados hasta ahora en la forma de un grafo
!git log --all --graph --decorate 

* commit 3c560a7efa7542a8fa07a543c8a66217e8494ec2 (HEAD -> master)
  Author: eddysanoli <eddysanoli@gmail.com>
  Date:   Wed Feb 10 21:44:53 2021 -0600
  
      Jupyter notebook de la tarea no 2


In [11]:
# Se "stagea" el archivo de la tarea 1
!git add "Tareas/Tarea No. 1 - Herramientas.docx"

In [12]:
# Nuevo commit con los archivos de la tarea 1
# El mensaje que acompaña al commit estará incorrecto y 
# mencionará la tarea 2 en lugar de la 1. Esto se solucionará después.
!git commit

[master 4c20e71] Resolucion de tarea no 2
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 Tareas/Tarea No. 1 - Herramientas.docx


In [14]:
# Se corrige el mensaje del commit
!git commit --amend

[master 1fdfecd] Resolucion de tarea no 1
 Date: Wed Feb 10 21:47:52 2021 -0600
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 Tareas/Tarea No. 1 - Herramientas.docx


In [15]:
# Se muestra la nueva historia de commits
!git log --all --graph --decorate 

* commit 1fdfecd48ac11e6e2436bdd43b8b08f6d2a9cc39 (HEAD -> master)
| Author: eddysanoli <eddysanoli@gmail.com>
| Date:   Wed Feb 10 21:47:52 2021 -0600
| 
|     Resolucion de tarea no 1
| 
* commit 3c560a7efa7542a8fa07a543c8a66217e8494ec2
  Author: eddysanoli <eddysanoli@gmail.com>
  Date:   Wed Feb 10 21:44:53 2021 -0600
  
      Jupyter notebook de la tarea no 2


In [16]:
# Se crea la rama "literatura"
!git branch "Literatura"

In [18]:
# Se muestran las branches
!git branch
!git log --all --graph --decorate 

  Literatura
* master
* commit 1fdfecd48ac11e6e2436bdd43b8b08f6d2a9cc39 (HEAD -> master, Literatura)
| Author: eddysanoli <eddysanoli@gmail.com>
| Date:   Wed Feb 10 21:47:52 2021 -0600
| 
|     Resolucion de tarea no 1
| 
* commit 3c560a7efa7542a8fa07a543c8a66217e8494ec2
  Author: eddysanoli <eddysanoli@gmail.com>
  Date:   Wed Feb 10 21:44:53 2021 -0600
  
      Jupyter notebook de la tarea no 2


In [21]:
!git status

On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.gitignore
	.vscode/
	Lectures/
	Literatura/
	Tareas/Tarea No. 1 - Herramientas.pdf

nothing added to commit but untracked files present (use "git add" to track)


In [22]:
# Se agrega un archivo de .gitignore para obviar las carpetas
# ".vscode", "Lectures" y "Literatura"
!git add ".gitignore"

In [23]:
# Se hace el
!git commit

[master ee16139] Adicion de archivo "gitignore"
 1 file changed, 3 insertions(+)
 create mode 100644 .gitignore


In [25]:
# Luego del commit ya no se considera a las carpetas 
# ".vscode", "Lectures" y "Literatura" como "untracked files"
!git status

On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Tareas/Tarea No. 1 - Herramientas.pdf

nothing added to commit but untracked files present (use "git add" to track)


In [26]:
# Se cambia a la branch "Literatura"
!git checkout Literatura

Switched to branch 'Literatura'


In [27]:
# Se hace un commit con la versión pdf de la tarea 1
!git add "Tareas/Tarea No. 1 - Herramientas.pdf"
!git commit

[Literatura ff922a4] PDF de la tarea no 1
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 Tareas/Tarea No. 1 - Herramientas.pdf


In [28]:
# Se observa como el commit se realizó en la branch "Literatura"
!git log --all --graph --decorate 

* commit ff922a48d56d60f3a5f46b5c6c991180a99e3818 (HEAD -> Literatura)
| Author: eddysanoli <eddysanoli@gmail.com>
| Date:   Wed Feb 10 22:11:47 2021 -0600
| 
|     PDF de la tarea no 1
|   
| * commit ee161392274a0bced502820b1ae66bbd9180e18a (master)
|/  Author: eddysanoli <eddysanoli@gmail.com>
|   Date:   Wed Feb 10 22:06:23 2021 -0600
|   
|       Adicion de archivo "gitignore"
| 
* commit 1fdfecd48ac11e6e2436bdd43b8b08f6d2a9cc39
| Author: eddysanoli <eddysanoli@gmail.com>
| Date:   Wed Feb 10 21:47:52 2021 -0600
| 
|     Resolucion de tarea no 1
| 
* commit 3c560a7efa7542a8fa07a543c8a66217e8494ec2
  Author: eddysanoli <eddysanoli@gmail.com>
  Date:   Wed Feb 10 21:44:53 2021 -0600
  
      Jupyter notebook de la tarea no 2


In [30]:
# Se agrega el archivo ".gitignore" en esta branch
!git add ".gitignore"
!git commit

[Literatura 38909d7] Archivo .gitignore de la branch "Literatura"
 1 file changed, 3 insertions(+)
 create mode 100644 .gitignore


In [32]:
# Se regresa a la master branch
!git checkout master

Switched to branch 'master'


In [1]:
# Se "linkea" el repositorio creado en Github
!git remote add origin https://github.com/eddysanoli-galileo/Data-Science-en-Python.git

error: remote origin already exists.


In [2]:
# Se hace un push de todas las ramas y contenidos del
# repositorio a Github.
!git push "https://github.com/eddysanoli-galileo/Data-Science-en-Python.git" --all

To https://github.com/eddysanoli-galileo/Data-Science-en-Python.git
 * [new branch]      Literatura -> Literatura
 * [new branch]      main -> main
